# Dataset Generation

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import math
import shutil
from datetime import datetime
from math import exp
from random import seed
from random import random
from scipy.spatial import distance

import sys
sys.path.append('../common/')

from rbf_tools import *
from optimisation import *
from one_dim import *
from two_dim import *
from final import calculate_shape


In [5]:
def generate_dataset(data_per_sample, ratio_of_points, n_dim, lrs, bs, u_exacts, N, n=10):
    p_ref=np.zeros(n_dim)
    nunsuccessful = 0
    dataset = []
    a = 0 # lower limit of hypercube
    for lr_init, b in zip(lrs,bs):
        for u_exact in u_exacts:
                print(f"Dimension: {n_dim}")
                for i in range(data_per_sample):
                    x_axis = sample_from_hypercube(n_dim,low=a, high=b, n=N)
                    x_axis = sorted( x_axis, key = lambda x: np.linalg.norm(x - p_ref ) )
                    x_axis = np.reshape(x_axis, (N,n_dim)) 
    
                    #number_points= x_axis.shape[0]
    
                    distance = get_distance_matrix(x_axis)
                    
                    closest = np.argsort(distance, axis=1)
                    index_tot=[]
                    for i in range(ratio_of_points):
                        x_local=np.zeros((n,2))
                        for j in range(n):
                            x_local[j]=x_axis[closest[i][j]]
                            index_sorted=sorted(closest[i][:n])
                        if index_sorted not in index_tot:
                            index_tot.append(index_sorted)
                            if n_dim==1:
                                x_local[:,1]=0
                            
                            eps_init = initial_guess(x_local)
                            L =  get_int_matrix(x_local,eps_init)
                            cond_init = np.linalg.cond(L,'fro')
                            eps_final,nunsuccessful=calculate_shape(x_local,lr_init)
    
                            if nunsuccessful==0:
                                L =  get_int_matrix(x_local,eps_final)
                                cond = np.linalg.cond(L,'fro')
                                print('sucess')
                                X_data=x_local.flatten()
                                dataset.append([X_data, eps_final])
    
    return dataset

In [3]:
# 1-d
data_per_sample=700
ratio_of_points=1
n_dim=1
lrs=[0.1,0.1,0.1]
bs=[0.01,0.1,1.0]
N=10
n=10
freq = 2/0.1
u_exacts = [lambda x: np.exp(np.sin(np.pi*x)), lambda x: 1/(1+16*(x**2)), lambda x: np.cos(freq*np.pi*x) ]
dataset = generate_dataset(data_per_sample,ratio_of_points,n_dim,lrs,bs,u_exacts,N,n=n)
pickle.dump(dataset, open("sort_1d_11_11.5_10.pkl",'wb'))
print(dataset)

Dimension: 1
update size: 0.16999605248826957
loss size: 6.80851922331485
update size: 0.04078425166236765
loss size: 5.661252350549979
sucess
update size: 0.007771516388643831
loss size: 6.058567684851713
update size: 0.010091414667913303
loss size: 5.934145421096332
update size: 0.0438492215332068
loss size: 4.521844623411539
sucess
update size: 0.006698415748078081
loss size: 5.816067988477744
sucess


KeyboardInterrupt: 

In [6]:
# 2-d
data_per_sample=50
ratio_of_points=20
n_dim=2
lrs=[0.1,0.1,0.1,0.1]
bs=[0.001,0.01,0.1,1.0]
N=20
n=10
freq = 2/0.1
u_exacts =  [lambda x,y,alpha=0.1: (1+np.exp(-1.0/alpha)-np.exp(-x/alpha)-np.exp((x-1.0)/alpha))*(1+np.exp(-1.0/alpha)-np.exp(-y/alpha)-np.exp((y-1.0)/alpha)),\
             lambda x,y,alpha=1.0: (1+np.exp(-1.0/alpha)-np.exp(-x/alpha)-np.exp((x-1.0)/alpha))*(1+np.exp(-1.0/alpha)-np.exp(-y/alpha)-np.exp((y-1.0)/alpha)),\
             lambda x,y: 0.75*np.exp(-(9.0*x-2.0)**2/4.0-(9.0*y-2.0)**2/4.0)+0.75*np.exp(-(9.0*x+1.0)**2/49.0-(9.0*y+1.0)**2/10.0)+0.5*np.exp(-(9.0*x-7.0)**2/4.0-(9.0*y-3.0)**2/4.0)-0.2*np.exp(-(9.0*x-4.0)**2-(9.0*y-7.0)**2)]
dataset = generate_dataset(data_per_sample,ratio_of_points,n_dim,lrs,bs,u_exacts,N,n=n)
pickle.dump(dataset, open("sort_2d_11_11.5_10.pkl",'wb'))
print(dataset)    

Dimension: 2
sucess
sucess
sucess
sucess
sucess
sucess
sucess
sucess
sucess
sucess


KeyboardInterrupt: 